In [18]:
import os
import json
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path

import wandb
from dotenv import load_dotenv
import numpy as np
import polars as pl
from kaggle.api.kaggle_api_extended import KaggleApi
from kagglesdk.competitions.types.competition_api_service import SubmissionGroup

## Config

In [19]:
load_dotenv(dotenv_path="../../.env")


@dataclass
class Config:
    RUN_ID = "hc-058"
    MESSAGE = "Good Luck!"


cfg = Config()

RUNS = Path("../../runs") / cfg.RUN_ID
OUTPUT = Path("../../output")
INPUT = Path("../../input")

MANIFEST_PATH = RUNS / "manifest.json"
SUB_PATH = OUTPUT / f"{cfg.RUN_ID}.csv"

api = KaggleApi()
api.authenticate()

with open(MANIFEST_PATH) as f:
    m = json.load(f)

wandb.login(key=os.environ.get("WANDB_API_KEY"))
wandb_project = os.environ.get("COMPETITION_NAME")
WANDB_ID = m["wandb_id"]

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/hanse/.netrc


## Submit

In [20]:
test = np.load(RUNS / "test.npy")[:250000]

sample_sub = pl.read_csv(INPUT / "sample_submission.csv")

if len(sample_sub) != len(test):
    raise ValueError(
        f"Row mismatch: sample={len(sample_sub)}, test={len(test)}"
    )

sub = sample_sub.with_columns(
    pl.Series("y", test)
)
sub.write_csv(SUB_PATH)

result = api.competition_submit(
    file_name=SUB_PATH,
    message=cfg.MESSAGE,
    competition=wandb_project
)

ref = result.ref

submitted_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Ref: {ref}")

100%|██████████| 4.74M/4.74M [00:01<00:00, 2.94MB/s]


Ref: 47217091


# Public Score

In [21]:
# === Check Public Score ===
subs = api.competition_submissions(
    wandb_project,
    group=SubmissionGroup.SUBMISSION_GROUP_SUCCESSFUL
)

target = next((s for s in subs if s.ref == ref), None)

if target:
    public = target.public_score
    print(f"Public LB: {public}")
else:
    print("Subs not found")

Public LB: 0.97747Date: 2025-10-06 11:19:26.050000


In [22]:
# === Record Public Score in WandB===
run = wandb.init(
    project=wandb_project,
    id=WANDB_ID,
    resume="must",
    dir="../../artifacts"
)
run.summary["public"] = public

run.config.update({"ref": ref})

run.finish()

auc_oof,0.97668
public,0.97747


In [23]:
# === Update Manifest ===
m["submission"]["ref"] = ref
m["submission"]["file"] = str(SUB_PATH)
m["submission"]["competition"] = wandb_project
m["submission"]["public_score"] = public
m["submission"]["submitted_at"] = submitted_at

with open(MANIFEST_PATH, "w") as f:
    json.dump(m, f, indent=4)

# Private Score

In [24]:
subs = api.competition_submissions(
    wandb_project,
    group=SubmissionGroup.SUBMISSION_GROUP_SUCCESSFUL
)

target = next((s for s in subs if s.ref == ref), None)

private = target.private_score

if target:
    print(f"Private LB: {private}")
else:
    print("Subs not found")

Private LB: 0.97712


In [25]:
# === Record Private Score in WandB===
run = wandb.init(
    project=wandb_project,
    id=WANDB_ID,
    resume="must",
    dir="../../artifacts"
)
run.summary["private"] = private

run.finish()

auc_oof,0.97668
private,0.97712
public,0.97747


In [26]:
# === Update Manifest ===
m["submission"]["private_score"] = private
with open(MANIFEST_PATH, "w") as f:
    json.dump(m, f, indent=4)

In [27]:
from cuml.linear_model import Ridge

In [28]:
help(Ridge)

Help on class Ridge in module cuml.linear_model.ridge:

class Ridge(cuml.internals.base.Base, cuml.internals.interop.InteropMixin, cuml.internals.mixins.RegressorMixin, cuml.linear_model.base.LinearPredictMixin, cuml.internals.mixins.FMajorInputTagMixin)
 |  Ridge(
 |      *,
 |      alpha=1.0,
 |      solver='auto',
 |      fit_intercept=True,
 |      normalize=False,
 |      handle=None,
 |      output_type=None,
 |      verbose=False
 |  )
 |
 |  Ridge extends LinearRegression by providing L2 regularization on the
 |  coefficients when predicting response y with a linear combination of the
 |  predictors in X. It can reduce the variance of the predictors, and improves
 |  the conditioning of the problem.
 |
 |  cuML's Ridge can take array-like objects, either in host as
 |  NumPy arrays or in device (as Numba or `__cuda_array_interface__`
 |  compliant), in addition to cuDF objects. It provides 3
 |  algorithms: SVD, Eig and CD to fit a linear model. In general SVD uses
 |  signific